In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import os

print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

In [ ]:
from sklearn.linear_model import LinearRegression,Ridge,Lasso,ElasticNet
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn import model_selection
from sklearn.model_selection import KFold

In [ ]:
from sklearn.feature_selection import RFE
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score as acc
from sklearn.metrics import r2_score
from sklearn import ensemble

In [ ]:
df = pd.read_csv("../input/kc-housesales-data/kc_house_data.csv")

We have read our data set

In [ ]:
df.head(5)

To print five rows of our data set

In [ ]:
df.shape

There are 21597 rows and 21 columns in our data set

In [ ]:
df.columns

Prints us the number of columns in our data set.

In [ ]:
df.info()

This explains us about the type of variables.

In [ ]:
df.describe()

This describes the data in a stastical way

In [ ]:
df.isnull().sum()

There is no null values in our data set.

In [ ]:
df.bedrooms.value_counts()

This shows us the count of houses based on bed rooms.

To represent the above data graphically

In [ ]:
sns.countplot(df.bedrooms)

It's very clear that the 3 bed room houses are more in our given data set.

In [ ]:
df.floors.value_counts()

To count the houses based on the number of floors.

In [ ]:
sns.countplot(df.floors)

This shows there are 10673 houses which has only first floor.

In [ ]:
df.grade.value_counts()

In [ ]:
sns.countplot(df.grade)

This shows us the grade of the houses.

In [ ]:
sns.boxplot(df.grade,df.price)

Houses with higher grade has higher price value.

In [ ]:
sns.regplot(df.grade,df.price)

The split of data and their reg plot.

In [ ]:
sns.distplot(df.price)

This shows that our target variable price is right skewed.

We have to convert our right skewed target variable into normal distribution.

In [ ]:
df['price'] = df['price'].apply(lambda x : np.log(x))

In [ ]:
sns.distplot(df.price)

Now our data is normally distributed

In [ ]:
continous_columns=['id','price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'waterfront','sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15']
for i in continous_columns:
    print("columns name:",i)
    sns.distplot(df[i])
    plt.show()

We have printed the different Histograms of our varibales.

In [ ]:
for i in continous_columns:
    print("column name:",i)
    sns.boxplot(df[i])
    plt.show()

Box plot of our variables which clearly shows us the outliers present in it.

In [ ]:
mod=[]
def find_outlier(df_in, col_name):
    q1 = df_in[col_name].quantile(0.25)
    q3 = df_in[col_name].quantile(0.75)
    iqr = q3-q1 #Interquartile range
    fence_low  = q1-1.5*iqr
    fence_high = q3+1.5*iqr
    df_high=df_in.loc[(df_in[col_name] > fence_high)]
    #df_out = df_in.loc[(df_in[col_name] > fence_low) & (df_in[col_name] < fence_high)]
    outlier_percentage=(df_high.shape[0]/len(df))*100
    outlier_percentage=round(outlier_percentage,2)
    print(outlier_percentage)
    mod.append((col_name,outlier_percentage))

In [ ]:
mod

In [ ]:
for i in continous_columns:
    find_outlier(df,i)

In [ ]:
length=len(continous_columns)

In [ ]:
result=[]
names=[]
for i in range(0,length-1):
    result.append(mod[i][1])
    names.append(mod[i][0])

In [ ]:
    names

In [ ]:
p=pd.DataFrame(result)
p.rename(columns={0:"outlier_percentage"},inplace=True)

In [ ]:
p.plot.bar(figsize=(20, 8))
plt.xticks(np.arange(length),names)
plt.title("percentage of outliers in each column")
plt.axhline(5)#axis 
plt.show()

We here by print the Outliers present in our data set.

In [ ]:
df.drop(['date'],axis=1,inplace=True)

As we don't need date column we are droping it.

In [ ]:
df.corr()

In [ ]:
df.cov()

In [ ]:
fig,ax= plt.subplots()
fig.set_size_inches(11.7, 8.27)
sns.heatmap(df.corr(),annot=True,linewidths=0.2,vmax = .9)
bottom, top = ax.get_ylim()
ax.set_ylim(bottom + 0.5, top - 0.5)

Fitting our Base model.

In [ ]:
y=df['price']
X=df.drop('price',axis=1)
xc=sm.add_constant(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [ ]:
import scipy.stats as stats
print(np.abs(round(stats.norm.isf(q = 0.025),2)))

In [ ]:
lin_reg = LinearRegression()
model = lin_reg.fit(X_train,y_train)
print(f'R^2 score for train: {lin_reg.score(X_train, y_train)}')
print(f'\nR^2 score for test: {lin_reg.score(X_test, y_test)}')

In [ ]:
from scipy.stats import ttest_1samp
print(df.mean(),np.std(df,ddof = 1))

In [ ]:
Xc = sm.add_constant(X)
lin_reg = sm.OLS(y,Xc).fit()
lin_reg.summary()

Our basr mode is fitted with the 0.77 R^2 value.

In [ ]:
lin_reg = LinearRegression()
lin_reg.fit(X, y)

print(f'Coefficients: {lin_reg.coef_}')
print(f'\nIntercept: {lin_reg.intercept_}')
print(f'\nR^2 score: {lin_reg.score(X, y)}')

In [ ]:
params = {'n_estimators': 500, 'max_depth': 4, 'min_samples_split': 2,
          'learning_rate': 0.01, 'loss': 'ls'}
model = ensemble.GradientBoostingRegressor(**params)

model.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
model_score = model.score(X_train,y_train)
# Have a look at R sq to give an idea of the fit ,
# Explained variance score: 1 is perfect prediction
print('R2 sq: ',model_score)
y_predicted = model.predict(X_test)

# The mean squared error
print("Mean squared error: %.2f"% mean_squared_error(y_test, y_predicted))
# Explained variance score: 1 is perfect prediction
print('Test Variance score: %.2f' % r2_score(y_test, y_predicted))
print("RMSE:%.2f"% np.sqrt(mean_squared_error(y_test, y_predicted)))

In [ ]:
print(f'R^2 score for train: {model.score(X_train, y_train)}')
print(f'\nR^2 score for test: {model.score(X_test, y_test)}')

In [ ]:
from sklearn.model_selection import cross_val_predict

fig, ax = plt.subplots()
ax.scatter(y_test, y_predicted, edgecolors=(0, 0, 0))
ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=4)
ax.set_xlabel('Actual')
ax.set_ylabel('Predicted')
ax.set_title("Actual vs Predicted")
plt.show()